In [1]:
import pandas as pd

## <a name="find_non_matched_rows"></a>Find rows in a source DataFrame that do not have a match in a second DataFrame

If you have a source DataFrame and want to check whether a second DataFrame has a corresponding row, you can use a left merge with the source on the lefthand side. A left merge returns all of the merge keys from the lefthand DataFrame regardless of whether a matching key is found on the right. When there is no match, the columns from the right DataFrame are filled with NaN.

A common use case for this merge is for finding files that require post-processing. If you have a set of source filenames (e.g. '.wav' files) in one DataFrame and a set of existing post-processed filenames (e.g. '.fb' files from the `formant` command) in another, you can use the merge described in this section to find the source filenames that don't have a matching post-processed file. These are the ones that require post-processing.

For this example the source DataFrame will be the left DataFrame `ldf`. It contains a list of '.wav' filenames, parsed into a barename and extension.

In [48]:
ldf = pd.DataFrame.from_records([
    ('file1.wav', 'file1', '.wav'),
    ('file2.wav', 'file2', '.wav'),
    ('file3.wav', 'file3', '.wav')
], columns=['fname', 'barename', 'ext'])
ldf

,fname,barename,ext
0,file1.wav,file1,.wav
1,file2.wav,file2,.wav
2,file3.wav,file3,.wav


The `rdf` DataFrame has a corresponding set of filenames that have the same barenames as `ldf` with different extensions. Notice that it does not contain a filename that corresponds to the 'file2' barename in `ldf`. It does contain a barename not found in `ldf` ('file4').

In [49]:
rdf = pd.DataFrame.from_records([
    ('file1.fb', 'file1', '.fb'),
    ('file3.fb', 'file3', '.fb'),
    ('file4.fb', 'file4', '.fb')
], columns=['fname', 'barename', 'ext'])
rdf

,fname,barename,ext
0,file1.fb,file1,.fb
1,file3.fb,file3,.fb
2,file4.fb,file4,.fb


Our goal is to find each '.wav' file in `ldf` that does not have a corresponding '.fb' file in the second Dataframe `rdf`.

Performing a left merge on `ldf` preserves all of its key values, in this case 'barename'. This means that each barename value in `ldf` is represented by at least one row in the output. When there is no matching key from the right DataFrame, then the columns contributed from the right are filled with NaN. Notice that the 'file4' barename from the right DataFrame is not in the merge result. A left merge does not preserve all key values from the right DataFrame.

In [31]:
mdf = ldf.merge(rdf, on='barename', how='left', suffixes=['_lt', '_rt'])
mdf

,fname_lt,barename,ext_lt,fname_rt,ext_rt
0,file1.wav,file1,.wav,file1.fb,.fb
1,file2.wav,file2,.wav,NaN,NaN
2,file3.wav,file3,.wav,file3.fb,.fb


To find all of the '.wav' files that do not have a corresponding '.fb' file, select the rows from the merged DataFrame where the 'ext_rt' column has a value of NaN.

In [32]:
mdf[mdf.ext_rt.isna()]

,fname_lt,barename,ext_lt,fname_rt,ext_rt
1,file2.wav,file2,.wav,NaN,NaN


***Important*** Be careful! If your right DataFrame contains files you don't expect, you could be in trouble. In `rdf2` there are two '.txt' files in addition to the '.fb' files.

In [36]:
rdf2 = pd.DataFrame.from_records([
    ('file1.fb', 'file1', '.fb'),
    ('file2.txt', 'file2', '.txt'),
    ('file3.wav', 'file3', '.fb'),
    ('file3.txt', 'file3', '.txt')
], columns=['fname', 'barename', 'ext'])
rdf2

,fname,barename,ext
0,file1.fb,file1,.fb
1,file2.txt,file2,.txt
2,file3.wav,file3,.fb
3,file3.txt,file3,.txt


Merging `rdf2` with `ldf` does not produce the result we want! Since we only match on barename values, the existence of 'file2.txt' masks the fact that 'file2.fb' is missing, and there are no NaN values in the merge result.

(Secondarily, the barename 'file3' matches twice, once each for the '.fb' and '.txt' file.)

In [38]:
mdf2 = ldf.merge(rdf2, on='barename', how='left', suffixes=['_lt', '_rt'])
mdf2

,fname_lt,barename,ext_lt,fname_rt,ext_rt
0,file1.wav,file1,.wav,file1.fb,.fb
1,file2.wav,file2,.wav,file2.txt,.txt
2,file3.wav,file3,.wav,file3.wav,.fb
3,file3.wav,file3,.wav,file3.txt,.txt


To fix this problem, ensure that the right DataFrame does not contain any files that are not relevant to the task. In our case, we want to find '.wav' files that do not have a matching '.fb' file, so we take a subset of `rdf2` that contains only '.fb' files.

The subset looks like this:

In [50]:
rdf2[rdf2.ext == '.fb']

,fname,barename,ext
0,file1.fb,file1,.fb
2,file3.wav,file3,.fb


Using this subset in the merge produces the intended result, where 'file2' has a NaN value in the 'ext_rt' column.

In [41]:
mdf2 = ldf.merge(
    rdf2[rdf2.ext == '.fb'],   # Subset of '.fb' files
    on='barename',
    how='left',
    suffixes=['_lt', '_rt']
)
mdf2

,fname_lt,barename,ext_lt,fname_rt,ext_rt
0,file1.wav,file1,.wav,file1.fb,.fb
1,file2.wav,file2,.wav,NaN,NaN
2,file3.wav,file3,.wav,file3.wav,.fb
